# Homework #1: Bandit algorithms

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!pip -q install git+https://github.com/rlberry-py/rlberry.git@v0.3.0#egg=rlberry[default] 

In [ ]:
from rlberry.agents.bandits import BanditWithSimplePolicy
from rlberry.wrappers import WriterWrapper

## Problem #1: Gaussian bandits

Consider the bandit problem where each arm follows normal distribution with its own mean and variance $\mathcal{D}_a \sim \mathcal{N}(\mu_a, \sigma^2_a)$, where both parameters $\mu_a$ and $\sigma^2_a$ may differs for all arms.

In [ ]:
# Environment, documentation: https://rlberry.readthedocs.io/en/latest/generated/rlberry.envs.bandits.NormalBandit.html
from rlberry.envs.bandits import NormalBandit

Basically, the task follows:
- Implement UCB-1, Thompson sampling and KL-UCB algorithms for this problem;
- Study different properties of these algorithms.

### UCB-1
All the theory for this algorithm was given, you suppose to prove that it also has nearly optimal regret even for sub-gaussian distribution (thus for gaussian too). Feel free to change method signature as you want. 

**Task**: implement it.

In [ ]:
class UCBAgent(BanditWithSimplePolicy):
    """
        UCB-1 agent for gaussian bandits.

        Parameters:
        -----------
        sigma: sub-gaussian parameter
        env: rlberry bandit environment

    """
    name = "UCB-1"

    def __init__(self, env, sigma, **kwargs):
        """
            Usefull fields of base class:
            * self.n_arms : number of arms
        """
        BanditWithSimplePolicy.__init__(self, env, **kwargs)
        # To track statistics on chosen actions
        self.env = WriterWrapper(self.env, self.writer, write_scalar="action")
        # TODO

    def fit(self, budget=None, **kwargs):
        """
        Fit function for greedy agent
        Parameters
        ----------
        budget: int
            Total number of iterations, also called horizon.
        """
        # TODO

### Thompson sampling

Now we consider the algorithm that is not based on Optimism in the face of uncertainty principle but can be very efficient on many environments.

Let us fix one arm $a$. Then rewards $r$ follows some parametric distribution $p(r | \theta_a)$, where the family of distribution is the same for all arms. Let us consider some \textit{prior} distribution over parameters $\rho(\theta_a)$. Given sampled rewards $r_1,\ldots, r_n$, we may define a posterior distribution of parameter $\theta_a$ as follows
$$
    p(\theta | r_1,\ldots,r_n) = \frac{\prod_{i=1}^n p(r_i | \theta) \rho(\theta)}{\int_{} \prod_{i=1}^n p(r_i | \theta) \rho(\theta) d \theta}
$$
Then Thompson Sampling (TS) algorithm propose the following:
- Sample $\theta^t_a$ from the postrior distribution for all arms $a \in \mathcal{A}$;
- Select $\hat a^t = \arg\max_{a} \mathbb{E}[r | \theta^t_a ] = \arg\max_{a} \int r p(r | \theta^t_a) d r$, in other words, we find the optimal arm while treating sampled parameters as real one.
- Update the posterior distribution for arm $\hat a^t$ given new reward $r_t$.



Usually, computation of posterior distribution is intractable, however, if we have conjugate prior, we may efficiently compute it. For this particular problem reward follows normal distribution with parameters $\mu_a, \sigma^2_a$. It is well-known that  there is a conjugate prior for normal distribution, so, TS for gaussian bandits is tractable.

**Task**: implement TS algorithm for two cases:
- Assume that variance for all distributions of arms is given. In this case, you may assume Bayessian model only for mean $\mu_a$ and define prior as normal distribution.
- Implement variance-adaptive version of TS algorithm. In this case you have two parameters for Bayesian model, for which you have normal-inverse-gamma distribution https://en.wikipedia.org/wiki/Normal-inverse-gamma_distribution 

Parameters of prior distribution should be treated as hyperparameters and study dependence of them on performance of obtained algorithms.

In [ ]:
class TSNormalAgent(BanditWithSimplePolicy):
    """
        TS agent for Gaussian bandits with known variance.

        Parameters:
        -----------
        params: parameters of prior 
        env: rlberry bandit environment

    """
    name = "TS"

    def __init__(self, env, params, **kwargs):
        """
            Usefull fields of base class:
            * self.n_arms : number of arms
        """
        BanditWithSimplePolicy.__init__(self, env, **kwargs)
        # To track statistics on chosen actions
        self.env = WriterWrapper(self.env, self.writer, write_scalar="action")
        # TODO

    def fit(self, budget=None, **kwargs):
        """
        Fit function for greedy agent
        Parameters
        ----------
        budget: int
            Total number of iterations, also called horizon.
        """
        # TODO

In [ ]:
class TSNormalAgentVarianceAdaptive(BanditWithSimplePolicy):
    """
        TS agent for Gaussian bandits with unknown variance.

        Parameters:
        -----------
        params: parameters of prior 
        env: rlberry bandit environment

    """
    name = "AdaptiveTS"

    def __init__(self, env, prarams, **kwargs):
        """
            Usefull fields of base class:
            * self.n_arms : number of arms
        """
        BanditWithSimplePolicy.__init__(self, env, **kwargs)
        # To track statistics on chosen actions
        self.env = WriterWrapper(self.env, self.writer, write_scalar="action")
        # TODO

    def fit(self, budget=None, **kwargs):
        """
        Fit function for greedy agent
        Parameters
        ----------
        budget: int
            Total number of iterations, also called horizon.
        """
        # TODO

### KL-UCB

Finally, you are going to implement the last baseline that is called KL-UCB. This algorirthm is based on Optimism in the face of uncertainty principle, but in much more involved form. Basically, algorithm is following:

1) Warm-up: take each arm one time to obtain mean estimates;

2) Algorithm:
- Compute $\hat \mu^t(a)$ for all arms (just empirical estimate of the mean);
- Compute 
$$
    \tilde \mu^t(a) = \max_{q \in [0,1]} \{q : n^t(a) \cdot \mathrm{kl}(\hat \mu^t(a) || q) \leq \log(t) + c \log \log (t)\},
$$ where $\mathrm{kl}(a||b)$ is binary KL-divergence defined as $\mathrm{kl}(a||b) = a \log(a/b) + (1-a) \log((1-a)/(1-b))$.
- Play $\hat a^t = \arg\max_{a} \tilde \mu^t(a)$;
- Update all statistics.


The step of computation of $q$ requires binary search. In practice usually $c$ equals to $0$ whereas theory prescribed choose it as a very large constant. Unfortunately, this particular version of the algorithm works only for bounded distribution, which is not the case of Gaussian distribution.


However, as it described in Section 4 of the original KL-UCB paper https://arxiv.org/abs/1102.2490, this algorithm may be applied not only to bounded distributions, but also to so-called *single-parameter exponential family* algorithm, for which the (generalized) density has the following form
$$
    p(x | \theta) = \exp(x \cdot \theta - b(\theta) + c(x)).
$$
For example, normal distribution with fixed variance satisfies satisfies it. In this case we may replace binary KL divergence by
$$
    d(x, \mu(\theta)) = \sup_{\lambda} \left( \lambda x - \log \mathbb{E}_\theta[\lambda X] \right),
$$
where $\mu(\theta)$ is an expectation of $p(x|\theta)$ given parameter theta. In the case of single-parameter exponential familty you have one-to-one corresponce between $\mu(\theta)$ and $\theta$, so this function is well-defined.

**Task**.
- Implement KL-UCB for bounded distributions, check it convergence on Bernulli bandit case (seminar notebook). 
- Learn how to run bounded version of KL-UCB on any fixed interval (not only [0,1]) and apply it to Gaussian bandits by fixing some interval of your choice and applying clipping of reward;
- Compute $d(x, \mu(\theta))$ for Gaussian distributions in the closed form.
- Implement KL-UCB for Gaussian distributions (with fixed variance);

In [ ]:
class KLUCBBoundedAgent(BanditWithSimplePolicy):
    """
        KL-UCB agent for bounded bandits.

        Parameters:
        -----------
        segment: tuple (a,b) with the interval of truncation. By default (0,1)
        env: rlberry bandit environment

    """
    name = "TS"

    def __init__(self, env, segment=(0,1), **kwargs):
        """
            Usefull fields of base class:
            * self.n_arms : number of arms
        """
        BanditWithSimplePolicy.__init__(self, env, **kwargs)
        # To track statistics on chosen actions
        self.env = WriterWrapper(self.env, self.writer, write_scalar="action")
        # TODO

    def fit(self, budget=None, **kwargs):
        """
        Fit function for greedy agent
        Parameters
        ----------
        budget: int
            Total number of iterations, also called horizon.
        """
        # TODO

In [ ]:
class KLUCBGaussianAgent(BanditWithSimplePolicy):
    """
        KL-UCB agent for Gaussian bandits with known variance.

        Parameters:
        -----------
        sigma_sq: variance of arms
        env: rlberry bandit environment

    """
    name = "TS"

    def __init__(self, env, sigma_sq, **kwargs):
        """
            Usefull fields of base class:
            * self.n_arms : number of arms
        """
        BanditWithSimplePolicy.__init__(self, env, **kwargs)
        # To track statistics on chosen actions
        self.env = WriterWrapper(self.env, self.writer, write_scalar="action")
        # TODO

    def fit(self, budget=None, **kwargs):
        """
        Fit function for greedy agent
        Parameters
        ----------
        budget: int
            Total number of iterations, also called horizon.
        """
        # TODO

### Final Comparison

You have to do the following comparisons:

- Compare Thompson sampling with and without variance adaptation on Gaussian bandits with all different variance for all arms it two cases: there is no misspecification in variance (first TS algorithm have exact information) and there is misspecification (first TS algorithm obtains some upper bound on all variance). Study how much misspecification affects performance.
- Compare KL-UCB for bounded distributions with truncation and Gaussian one. Study dependence on the range of truncation on the performance of the algorithm.
- Finally, compare UCB-1, Thompson Sampling, and KL-UCB in the setting of known variance. Make conclusion.